In [1]:
import pandas as pd
import random
import re

In [2]:
# Read data from notes.tsv
data = pd.read_csv('notes.tsv', delimiter='\t')

In [3]:
# data = data.filter(regex='^(classification|summary)')
data = data.filter(regex='^(summary|misleading|notMisleading)')

In [4]:
import re

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Replace '&quot;' and '&amp;'
    text = text.replace('&quot;', '')
    text = text.replace('&amp;', '')

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove 'tweet', 'nnn', and 'notes'
    text = re.sub('(tweet|nnn|notes)', '', text)
    
    # Remove extra spaces
    text = re.sub(' +', ' ', text)
    
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    
    return text

data['summary'] = data['summary'].apply(preprocess_text)


In [5]:
data = data.drop_duplicates(subset='summary')

In [6]:
data['target'] = data.iloc[:, :-1].apply(lambda x: x.tolist(), axis=1)
data.drop(data.columns[:-2], axis=1, inplace=True)

In [ ]:
from langdetect import detect
from tqdm import tqdm

# Filter dataframe to keep only English summaries
tqdm.pandas()  # Enable tqdm progress bar

def detect_language(x):
    try:
        return detect(x) == 'en'
    except:
        return False

data['is_english'] = data['summary'].progress_apply(detect_language)
data = data[data['is_english']]
data.drop('is_english', axis=1, inplace=True)


In [8]:
data.to_csv('notes_cleaned.csv', sep=',', index=False)